# Cvičenie 8: Pravdepodobnosť výhry v pokri

Na dnešnom cvičení si ukážeme princípy objektovo orientovaného programovania na probléme predikcie výhry v hre poker, a to konkrétne vo variante Texas hold 'em. Cieľom v pokri je vykombinovať čo najsilnejšiu päticu kariet podľa definovaných pravidiel. V hold 'eme každý hráč dostane dve karty do rúk, a následne na stôl je postupne položených päť kariet, ktoré sú rovnaké pre každého hráča. Víťaz sa určí tak, že každý si vyberie najlepšiu kombináciu pätice kariet z dostupných sedem (päť na stole, dve v ruke) a tieto kombinácie sa porovnajú.

V Texas hold 'eme pritom sú uznávané nasledovné kombinácie (zostupne od najsilnejšieho):
 1. Royal Flush – postupka kariet 10 J Q K A v tej istej farbe
 2. Straight Flush – akákoľvek postupka piatich kariet rovnakej farby
 3. Four of a Kind – štvorica kariet s rovnakou hodnotou
 4. Full House – trojica a dvojica kariet s rovnakou hodnotou
 5. Flush – päť kariet tej istej farby
 6. Straight – postupka kariet rôznych farieb
 7. Three of a Kind – trojica kariet s rovnakou hodnotou
 8. Two Pairs – dve dvojice kariet s rovnakou hodnotou
 9. One Pair – dvojica kariet s rovnakou hodnotou
 10. High Card – ak ani jeden hráč nemá kombinácie, vyhrá ten s kartou s najvyššou hodnotou (karty sú zoradené 2 – A)

Ak teda máte Royal Flush, s istotou vyhráte. Ak dvaja hráči majú kombináciu na rovnakej úrovni, tiež sa rozhodne štýlom High Card. Napríklad ak máme dvoch hráčov s dvomi pármi 10 a 9, a jeden má navyše Q, kým druhý 8, tak vyhrá prvý (má vyššiu kartu). Keby okrem párov by obaja mali kartu s rovnakou hodnotou, nastane remíza.

Pre jednoduchosť úlohy budeme považovať iba jeden jediný okamih v pokri, a to moment, keď na stole už máme položené štyri karty a poznáme karty, ktoré držia v ruke hráči. Takto nám ostáva 44 možných výsledkov (na stôl bude položená jedna z nepoužitých kariet). Cieľom bude určiť pravdepodobnosť toho, že vyhrá hráč 1, hráč 2, alebo nastane remíza (v pokri dosť nepravdepodobné). Pravdepodobnosť výhry jednotlivých hráčov vieme vypočítať tak, že nasimulujeme všetky možné výsledky hry, t.j. kto vyhrá, ak na stôl položíme prvú dostupnú kartu? kto vyhrá ak na stôl položíme druhú dostupnú kartu? atď. Pri každom možnom výstupe obaja hráči vytvoria najsilnejšie možné kombinácie z piatich kariet na stole a z tých dvoch, ktoré držia v ruke. Následne sa porovnajú dve kombinácie hráčov, a vyhrá hráč, ktorý má silnejšiu kombináciu.

Samozrejme na reprezentáciu hernej situácie by sme mohli použiť aj natívne údajové štruktúry jazyka Python, nie je to ale úplne prírodzený spôsob. Práve preto si dnes vytvoríme vlastné reprezentácie, ktoré budú viac podobné tomu, ako rozmýšľame o tejto hre.

Konkrétne budeme pracovať s piatimi konceptmi alebo triedami:

* `Card` - reprezentuje kartu, ktorá má farbu a hodnotu;
* `Deck` - balík kariet;
* `Hand` - pätica kariet, definuje rôzne kombinácie pre poker;
* `Player` - hráč, ktorý drží v ruke dve karty;
* `Game` - samotná hra, ktorá zastrešuje celú funkcionalitu simulácie.

Dôrazom nebude samotné algoritmické riešenie, ale návrh objektového riešenia. Pred tým, než sa spustíme do samotnej implementácie, potrebujeme navrhnúť štruktúru, t.j. ktorá trieda bude zodpovedná za ktorú časť funkcionality. Pritom budeme implementovať nasledujúce funkcie pre riešenie jednotlivých podproblémov:

* `generate_setup` - pripraví hernú situáciu, určí ktoré karty budú mať hráči, a ktoré karty budú na stole;
* `is_royal_flush`, `is_straight_flush`, `is_four_of_a_kind`, `is_full_house`, `is_flush`, `is_straight`, `is_three_of_a_kind`, `is_two_pairs`, `is_pair` - zisťuje, či pätica kariet spĺňa podmienky jednotlivých kombinácií;
* `evaluate_hand` - vyhodnotí päticu kariet (nájde najsilnejšiu kombináciu a hodnotu relevantných kariet);
* `compare_highest_card` - porovná dve pätice kariet na základe najvyššej hodnoty karty;
* `find_better` - nájde lepšiu päticu kariet;
* `get_all_combinations` - vygeneruje zoznam všetkých možných kombinácií, ktoré hráč môže mať na základe dostupných kariet;
* `select_best` - nájde najsilnejšiu päticu, ktorú hráč môže vykombinovať z dostupných kariet;
* `calculate_chances` - vypočíta pravdepodobnosť jednotlivých možných výsledkov hry.

Ak máte hotový návrh, [stiahnite si kostru riešenia](sources/lab08/lab08.zip) a oboznámte sa s kódom. Zamerajte sa hlavne na členské premenné a na formu reprezentácie údajov.

## Hotové triedy

V riešení nájdete tri už úplne implementované triedy, a to `Card`, `Deck` a `Hand`.

`Card` je pomocná trieda pre reprezentáciu hracej karty. Kým bez vlastných tried by sme karty mohli reprezentovať ako dvojicu hodnôt farba (typu `string`) a hodnota (typu `int`), tieto hodnoty radšej ukladáme v členských premenných `suit` a `value` (rovnakých typov). Trieda ďalej definuje štandardné metódy `__str__`, ktorá vygeneruje stringovú reprezentáciu karty; `__eq__`, ktorá porovná dva objekty typu `Card`; a `__gt__`, ktorá tiež rieši porovnávanie, avšak nerovnosť dvoch kariet. Metóda `__str__` nám umožní použitie funkcie `print` nad kartou, metóda `__eq__` umožňuje porovnávanie kariet pomocou operátora `==`, a metóda `__gt__` zodpovedá operátoru `>`. Keďže trieda definuje dva operátory porovnávania, môžeme použiť aj ostatné (`>=`, `!=`, `<`, atď.).

Druhá trieda je `Deck`, ktorá definuje balík kariet (členská premenná `all_cards`), a zaznamenáva si, ktoré karty boli už použité (premenná `used_cards`), a ktoré ešte vieme použiť v rámci hry (`available_cards`). Ďalej definuje metódu `generate_setup`, ktorá vygeneruje hernú situáciu s dvomi hráčmi a štyrmi kartami na stole.

Ďalšia trieda, `Hand`, definuje päticu kariet, ktorú hráč vykombinuje z dostupných kariet. Pätica je reprezentovaná ako zoznam piatich objektov typu `Card`. Trieda definuje metódy pre vyhodnotenie pätice (`evaluate_hand` a `is_...`). Podobne ako v prípade kariet, aj tu máme definované funkcie `__eq__` a `__gt__` pre jednoduché porovnávanie pätíc.

Hotový je aj súbor `main.py`, ktorý definuje simuláciu hry.

## Trieda `Player`

V tomto kroku doimplementujeme triedu `Player`, teda triedu reprezentujúcu hráča. Trieda má jednu členskú premennú, `hand`, ktorá v sebe má zoznam dvoch kariet, ktoré hráč drží v ruke. Metóda `deal_hand` naplní túto premennú hodnotami na základe rozdávaných kariet.

**Úloha**: Implementujte metódu `get_all_combinations`, ktorá vygeneruje všetky možné pätice kariet hráča. Metóda má tri parametre, ktoré sú zoznamy kariet na stole. Návratová hodnota je zoznam pätíc kariet reprezentovaných ako `Hand`.

**Úloha**: Implementujte metódu `get_best_hand`, ktorá nájde a vráti najsilnejšiu päticu kariet (typu `Hand`). Metóda má rovnaké parametre ako `get_all_combinations`. Nezabudnite, že objekty typu `Hand` môžete porovnať pomocou štandardných operátorov.

## Trieda `Game`

Ostáva nám implementovať poslednú triedu `Game`, ktorá zastrešuje celkovú funkcionalitu. V konštruktore sa vytvorí balík kariet, a pridajú sa dvaja hráči. Implementujte chýbajúce metódy:

* `prepare_setup` - vygeneruje začiatočnú situáciu hry, a nastaví karty v rukách hráča (použite existujúce metódy ostatných tried); uložte aj zoznam kariet na stole.
* `find_winner` - nájde výhercu na základe karty `river`, ktorú dostane ako parameter. Metóda vráti 1 ak vyhrá hráč 1, 0 ak nastane remíza a -1 ak vyhrá hráč 2.
* `calculate_chances` - hlavná funkcia triedy, vypočíta pravdepodobnosť jednotlivých výsledkov hry a vráti tri hodnoty: pravdepodobnosť výhry hráča 1, pravdepodobnosť remízy, a pravdepodobnosť výhry hráča 2.

## Testovanie riešenia

Vaše riešenie môžete otestovať pomocou funkcie `main` v `main.py`.